In [46]:
import nltk
import sys
from nltk.corpus import brown
from __future__ import print_function
nltk.download('brown')

[nltk_data] Downloading package brown to /home/thefool/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [47]:
brown_tags_words = []
for sent in brown.tagged_sents():
    brown_tags_words.append(("START","START"))
    brown_tags_words.extend([(tag[:2],word) for (word,tag) in sent])
    brown_tags_words.append(("END","END"))

In [48]:
#loop corpus
cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words)
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords,nltk.MLEProbDist)

In [49]:
print("the probability of an adjective (JJ) being 'new' is",cpd_tagwords["JJ"].prob("new"))
print("the probability of an verb (VB) being 'duck' is",cpd_tagwords["VB"].prob("duck"))

the probability of an adjective (JJ) being 'new' is 0.0147234491763
the probability of an verb (VB) being 'duck' is 6.04271335094e-05


In [50]:
brown_tags = [tag for (tag,word) in brown_tags_words]

In [51]:
# count(t{i-1} ti)
#bigram的意思是 前后两个一组，联在一起
cfd_tags = nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
#p(ti|t{i-1})
cpd_tags = nltk.ConditionalProbDist(cfd_tags,nltk.MLEProbDist)

In [52]:
print("if we have just seen 'DT',the probability of 'NN' is",cpd_tags["DT"].prob("NN"))
print("if we have just seen 'VB',the probability of 'JJ' is",cpd_tags["VB"].prob("JJ"))
print("if we have just seen 'VB',the probability of 'NN' is",cpd_tags["VB"].prob("NN"))

if we have just seen 'DT',the probability of 'NN' is 0.505772252203
if we have just seen 'VB',the probability of 'JJ' is 0.0344348336527
if we have just seen 'VB',the probability of 'NN' is 0.10970977711


In [53]:
prob_tagsequence = cpd_tags["START"].prob("PP")*cpd_tagwords["PP"].prob("I")*\
    cpd_tags["PP"].prob("VB")*cpd_tagwords["VB"].prob("want")*\
    cpd_tags["VB"].prob("TO")*cpd_tagwords["TO"].prob("to")*\
    cpd_tags["TO"].prob("VB")*cpd_tagwords["VB"].prob("race")*\
    cpd_tags["VB"].prob("END")
                
print("the probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is",prob_tagsequence)

the probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is 1.08177664612e-14


In [54]:
distinct_tags = set(brown_tags)

In [55]:
sentence = ["I","want","to","race"]
sentlen = len(sentence)

In [56]:
viterbi = []

In [57]:
backpointer = []

In [58]:
first_viterbi = {}
first_backpointer = {}
for tag in distinct_tags:
    if tag == "START": continue
    first_viterbi[tag] = cpd_tags["START"].prob(tag)*cpd_tagwords[tag].prob(sentence[0])
    first_backpointer[tag] = "START"
print(first_viterbi)
print(first_backpointer)

{u'DO': 0.0, u'BE': 0.0, u'WD': 0.0, u'WQ': 0.0, u'FW': 0.0, u'IN': 0.0, u'HV': 0.0, u'VB': 0.0, u"''": 0.0, u')-': 0.0, u'WR': 0.0, u'JJ': 0.0, u'WP': 0.0, u'DT': 0.0, u',-': 0.0, u'NI': 3.3324520848931064e-07, u'PP': 0.014930900689060006, u'RP': 0.0, u'.-': 0.0, u'NN': 1.0580313619573935e-06, u')': 0.0, u'(': 0.0, u'*': 0.0, u',': 0.0, u'.': 0.0, u'TO': 0.0, u'RB': 0.0, u'NP': 1.7319067623793952e-06, u'RN': 0.0, u':': 0.0, u'PN': 0.0, u'``': 0.0, u'AB': 0.0, 'END': 0.0, u'CC': 0.0, u'CD': 0.0, u'*-': 0.0, u'AT': 0.0, u'(-': 0.0, u'CS': 0.0, u'NR': 0.0, u'MD': 0.0, u':-': 0.0, u"'": 0.0, u'AP': 0.0, u'--': 0.0, u'OD': 0.0, u'UH': 0.0, u'QL': 0.0, u'EX': 0.0}
{u'DO': 'START', u'BE': 'START', u'WD': 'START', u'WQ': 'START', u'FW': 'START', u'IN': 'START', u'HV': 'START', u'VB': 'START', u"''": 'START', u')-': 'START', u'WR': 'START', u'JJ': 'START', u'WP': 'START', u'DT': 'START', u',-': 'START', u'NI': 'START', u'PP': 'START', u'RP': 'START', u'.-': 'START', u'NN': 'START', u')': 'STAR

In [59]:
viterbi.append(first_viterbi)
backpointer.append(first_backpointer)

In [60]:
currbest = max(first_viterbi.keys(),key = lambda tag:first_viterbi[tag])
print ("Word","'"+sentence[0]+"'","current best two-tag sequence:",first_backpointer[currbest],currbest)

Word 'I' current best two-tag sequence: START PP


In [61]:
for wordindex in range(1,sentlen):
    this_viterbi = {}
    this_backpointer = {}
    prev_viterbi = viterbi[-1]
    
    for tag in distinct_tags:
        if tag =="START": continue
            
        best_previous = max(prev_viterbi.keys(),
                           key = lambda prevtag:prev_viterbi[prevtag]*cpd_tags[prevtag].prob(tag)*cpd_tagwords[tag].prob(sentence[wordindex]))
        
        this_viterbi[tag] = prev_viterbi[best_previous]*cpd_tags[best_previous].prob(tag)*cpd_tagwords[tag].prob(sentence[wordindex])
        
        this_backpointer[tag] = best_previous
        
    currbest = max(this_viterbi.keys(),key = lambda tag: this_viterbi[tag])
    print("Word","'"+sentence[wordindex]+"'","current best two-tag sequence:",this_backpointer[currbest],currbest)
        
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)

Word 'want' current best two-tag sequence: PP VB
Word 'to' current best two-tag sequence: VB TO
Word 'race' current best two-tag sequence: IN NN


In [62]:
prev_viterbi= viterbi[-1]
best_previous = max(prev_viterbi.keys(),
                   key = lambda prevtag: prev_viterbi[prevtag]*cpd_tags[prevtag].prob("END"))
best_tagsequece = ["END",best_previous]

backpointer.reverse()

In [63]:
current_best_tag = best_previous
for bp in backpointer:
    best_tagsequece.append(bp[current_best_tag])
    current_best_tag = bp[current_best_tag]

In [64]:
best_tagsequece.reverse()
print("the sentence was:",end = " ")
for w in sentence: print(w,end = " ")
print("\n")
print("the bast tag sequence is",end = " ")
for t in best_tagsequece: print(t,end = " ")
print("\n")
print("the probability of the best tag sequence is",prob_tagsequence)

the sentence was: I want to race 

the bast tag sequence is START PP VB IN NN END 

the probability of the best tag sequence is 1.08177664612e-14
